In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
def get_depot(url,hdepot,response='Other'):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get(str(url)+str(hdepot), headers=headers)
    content = r.content
    soup = BeautifulSoup(content)
    
    manufacturer = []
    prices = []
    describer = []
    responder = []

    items = soup.find_all('div', {'class':'package-pod__details-container  col__12-12'})
    for item in items:
        manufacturer.append(item.find('div', {'class': "pod-description"}).text.strip('\n').strip(' ').strip('\n'))
        for price in item.find_all('div', {'class': 'price'}):
            prices.append(''.join([price.strong.text.strip(), price.sup.text.strip()]))
        for describe in item.find_all('div', {'class': 'grid'}):
            describer.append(''.join([describe.strong.text.strip(), describe.sup.text.strip()]))
        for resp in item:
            responder = response
            
    
    
    items_info = zip(manufacturer,prices,describer,responder)
    return items_info

In [3]:
url1 = 'https://www.homedepot.com/p/sets/powertools/package/'
home_depot = ['Milwaukee/package_4','Dewalt/package_2','Ryobi/package_7','Rigid/package_11','Makita/package_14']

In [ ]:
n = len(home_depot)
csv_results = []
for i in range(n):
    csv_results.append(get_depot(url1, home_depot[i],'Tool'))

In [ ]:
def get_nordstrom(url,nord,response='Other'):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get(str(url)+str(nord), headers=headers)
    content = r.content
    soup = BeautifulSoup(content)
    
    manufacturer = []
    prices = []
    describer = []
    responder = []

    items = soup.find_all('article', {'class':'_1AOd3 QIjwE'})
    for item in items:
        manufacturer.append(item.find('div', {'class': "pod-description"}).text.strip('\n').strip(' ').strip('\n'))
        for price in item.find_all('div', {'class': 'YbtDD _18N5Q'}):
            prices.append(''.join([price.strong.text.strip(), price.sup.text.strip()]))
        for describe in item.find_all('h3', {'class': 'Dawzg _28b4r'}):
            describer.append(''.join([describe.strong.text.strip(), describe.sup.text.strip()]))
        for resp in item:
            responder = response
            
    
    
    items_info = zip(manufacturer,prices,describer,responder)
    return items_info

In [ ]:
url2 = 'https://www.nordstrom.com/browse/women/clothing/'
nordstrom = ['dresses?breadcrumb=Home%2FWomen%2FClothing%2FDresses&origin=topnav','jeans?breadcrumb=Home%2FWomen%2FClothing%2FJeans%20%26%20Denim&origin=topnav']

In [ ]:
n = len(nordstrom)
for i in range(n):
    csv_results.append(get_data(url2, nordstrom[i],'Fashion'))

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(csv_results),columns=['Manufacturer','Price','Description','Class'])
df.to_csv('products.csv', index=False, encoding='utf-8')

In [ ]:
df = pd.read_csv('products.csv', index_col = 0)
df.head

In [ ]:
df.replace(str(0), np.nan, inplace=True)
df.replace(0, np.nan, inplace=True)
df.dropna()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df, train_size = 0.7, random_state =42)

y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [ ]:
formula = 'Class ~ Manufacturer+Price+Description'

In [ ]:
model = smf.glm(formula = formula, data = x_train, family = sm.families.Binomial())
result = model.fit()

In [ ]:
predictions = result.predict(x_test)
predictions_nominal = ["Tool" if x < 0.5 else "Fashion" for x in predictions]
print(classification_report(y_test, predictions_nominal, digits = 3))

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors = 3)
pred = knn.fit(X_train, y_train).predict(X_test)

In [ ]:
print(confusion_matrix(y_test, pred).T)
print(classification_report(y_test, pred, digits=3))